In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from packages.preprocess import OurCONLLUDataset, load_raw_data

In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer, BertForTokenClassification
import numpy as np
import torch


In [6]:
datapath = '/cluster/projects/nn9851k/IN5550/norne-nb-in5550-train.conllu'
NORBERT = '/cluster/shared/nlpl/data/vectors/latest/216/'

In [8]:
import conllu

In [9]:
# loading raw data
print('Loading raw conllu')
con_df = load_raw_data(datapath)
condf, _ = train_test_split(
    con_df,
    test_size=0.9,
    random_state=1,
    shuffle=True,
    # stratify
)

# splitting data
print('Splitting data')
train_df, val_df = train_test_split(
    con_df,
    test_size=0.25,
    random_state=1,
    shuffle=True,
    # stratify
)

# creating data sets
print('Creating datasets/loaders')
train_dataset = OurCONLLUDataset(train_df)
val_dataset = OurCONLLUDataset(
    df=val_df,
    label_vocab=train_dataset.label_vocab
)

Loading raw conllu
Splitting data
Creating datasets/loaders


In [20]:
train_dataset.TokenList[1]

TokenList<Og, enda, har, jeg, ikke, nevnt, tribunesliting, på, Aspmyra, .>

In [23]:
a_token_list_item = train_dataset.TokenList[3]

In [24]:
for token in a_token_list_item:
    print(token['misc']['name'])

O
B-GPE_LOC
O
O
O
O
O
O


In [25]:
for item in a_token_list_item:
    if item['misc']['name'] == 'O':
        item['misc']['name'] = 'O-from-Per'

In [26]:
type(a_token_list_item)

conllu.models.TokenList

In [27]:
a_list= [a_token_list_item, a_token_list_item, a_token_list_item]

In [35]:
with open('my.conllu', 'w') as f:
    for i in a_list:
        f.write(i.serialize())


In [39]:
def pad_y(batch): # pass padding index
    return batch

In [8]:

def pad_y(batch): # pass padding index
    return batch
# creating data loaders
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn= pad_y)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=pad_y)
# train_loader = DataLoader(train_dataset, batch_size=32)
# val_loader = DataLoader(val_dataset, batch_size=32)

[jump](#JUMP)

In [9]:
tokenizer = BertTokenizer.from_pretrained(NORBERT)
bert = BertModel.from_pretrained(NORBERT)
bert_ftc = BertForTokenClassification.from_pretrained(
    NORBERT,
    num_labels=len(train_dataset.label_vocab),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at /cluster/shared/nlpl/data/vectors/latest/216/ were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification w

In [10]:
X, y = train_dataset[0]
# print(tokenizer(X)['input_ids'])
y

[16, 16, 16, 16, 16, 16, 16, 16, 16, 3, 0, 16, 16, 3, 0, 16, 16, 16]

In [11]:
X

'Frem til 6. april 2005 ble det ledet av Johannes Ulltveit-Moe, med Kristin Grimstad som medarbeider.'

In [12]:
from packages.preprocess import NewCONLLUDataset

In [13]:
new_train_dataset = NewCONLLUDataset(
    con_df,
    tokenizer,
)

In [14]:
new_valid_dataset = NewCONLLUDataset(
    con_df,
    tokenizer,
    new_train_dataset.label_vocab
)

In [15]:
new_train_dataset[0]

(tensor([  102,  2749,  4226,  4297, 26626,  5745, 17978, 11192, 18488, 18042,
          3473,  4958, 11969,  7414, 14170,  6083,  3693, 18488,  7017,  2857,
          3693, 18488,  4018,  7679,   103]),
 tensor([16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
         16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.]),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0]))

In [16]:
for i in range (4):
    print(len(new_train_dataset[i][0]))

25
37
33
19


In [31]:
def pad(batch): # pass padding index
    longest_sentence = max([X.size(0) for X, y, z in batch])

    new_X, new_y, new_z = [], [], []

    for X, y, z in batch:
        new_X.append(torch.nn.functional.pad(
            X,
            (0, longest_sentence - X.size(0)),
            value=0)
        )
        new_y.append(torch.nn.functional.pad(
            y,
            (0, longest_sentence - y.size(0)),
            value=0)
        )
        new_z.append(torch.nn.functional.pad(
            z,
            (0, longest_sentence - z.size(0)),
            value=0)
        )


    new_X = torch.stack(new_X).long()
    new_y = torch.stack(new_y).long()
    new_z = torch.stack(new_z).long()

    return new_X, new_y, new_z
# creating data loaders
new_train_loader = DataLoader(new_train_dataset, batch_size=32, 
collate_fn=pad)
new_val_loader = DataLoader(new_valid_dataset, batch_size=32, 
collate_fn=pad)

In [33]:
for batch in new_train_loader:
    X, y, z = batch
    # print('X:', X)
    # print('y:', y)
    # print('z:', z)
    
    print(bert_ftc(X, labels=y, attention_mask=z).loss.item())



3.050593137741089
3.0666370391845703
3.0953621864318848
3.0924265384674072
3.0740456581115723
3.0451440811157227
3.023669719696045
3.108832359313965


KeyboardInterrupt: 

In [28]:
len(x_tkn[1])

16

In [ ]:
tokenizer()

In [34]:
# train_dataset[9][1]

In [35]:
# label_vocab_len = 18

# def label_idx_to_tensor(label):
#     lbl_split = label
#     r = torch.zeros((len(lbl_split), label_vocab_len))
#     for i, ele in enumerate(lbl_split):
#         r[i][int(ele)] = 1
#     return r


In [36]:
# for i in range(4):
#     print(label_idx_to_tensor(train_dataset[i][1]))


# JUMP

In [8]:
# calling transformer model
print('Initializing model')
transformer = Transformer(
    NORBERT=NORBERT,
    num_labels=len(train_dataset.label_vocab),
    device='cpu',
    d_model=768,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048,
    dropout=0.2,
    activation='relu',
    epochs=10,
    lr_scheduler=False,
    factor=0.1,
    patience=2,
    loss_funct='binary-cross-entropy',
    random_state=1,
    verbose=True
)

Initializing model
Some weights of the model checkpoint at /cluster/shared/nlpl/data/vectors/latest/216/ were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTo

In [43]:

print('Fitting model')
transformer.fit(
    loader=train_loader,
    verbose=True
)


  0%|          | 0/10 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [46]:
from packages.model import BERT
# calling transformer model
print('Initializing model')
bert = BERT(
    NORBERT=NORBERT,
    num_labels=len(train_dataset.label_vocab), #TODO automize
    device='cpu',
    epochs=5, # tune
    pad_idx = train_dataset.label_indexer['O'],
    random_state=1,
    verbose=True
)


print('Fitting model')
bert.fit(
    loader=train_loader,
    verbose=True
)


Initializing model
Some weights of the model checkpoint at /cluster/shared/nlpl/data/vectors/latest/216/ were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTo

KeyboardInterrupt: 